In [ ]:
import numpy as np
import cv2
import sys

if len(sys.argv) < 2:
    print ('Enter path to image file')
    sys.exit()

img = cv2.imread('1.png')

output = img.copy()
#Convert image to Grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#Blur image
gray = cv2.GaussianBlur(gray, (3,3), 0)
#threshold
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#Canny edge detection
#edges = cv2.Canny(thresh, 50, 200)

cv2.imshow('thresh', thresh)

# # detect circles
circles = cv2.HoughCircles(image=gray,
                            method=cv2.HOUGH_GRADIENT, 
                            dp=1.1, 
                            param1=100,
                            param2=30, 
                            minDist=30, 
                            minRadius=10, 
                            maxRadius=60)

rcircles = np.uint16(np.around(circles))

if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    print ("Number of circles:", len(circles))

    # Count circles
    count=1 
    for (x, y, r) in circles:
        #Calculate radius in mm:
        r_mm = round(r/53.3, 2)
        # Create outer circle
        cv2.circle(output, (x,y), r, (0, 0, 0), 1)
        # Create center rectangle
        cv2.rectangle(output, (x-2, y-2), (x+2, y+2), (0,255,0), -1)
        # Add radius to center
        cv2.putText(output, str(r_mm), 
                    (x-15, y-5), 
                    cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                    0.7, (0, 0, 0), 1)
        # Print the radius of detected circles in pixels and mm
        print ('c' + str(count) + ' ' + str(r_mm) + ' ' + str(r))
        count += 1

    cv2.imshow("output", np.hstack([img, output]))
    cv2.waitKey(0)

Number of circles: 17
c1 0.96 51
c2 0.98 52
c3 1.11 59
c4 1.09 58
c5 0.92 49
c6 1.09 58
c7 0.94 50
c8 0.58 31
c9 0.77 41
c10 0.45 24
c11 0.51 27
c12 0.66 35
c13 0.53 28
c14 0.56 30
c15 0.53 28
c16 0.36 19
c17 0.28 15
